In [141]:
import itertools
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils 
from torch.autograd import Variable

from torchvision import transforms

import numpy as np
import pandas as pd

%matplotlib inline

In [142]:
df_label = pd.read_csv('./Platin_Data/mRNA_patient.csv', sep = '\t', index_col = 0)

In [143]:
df_label

,PLATINUM_STATUS
ID,
TCGA-61-1910,Sensitive
TCGA-24-0982,Resistant
TCGA-36-1580,Resistant
TCGA-25-1321,Sensitive
TCGA-30-1892,Resistant
...,...
TCGA-04-1347,Sensitive
TCGA-24-0968,Sensitive
TCGA-30-1891,Resistant


In [144]:
df_label.loc[df_label['PLATINUM_STATUS'] == 'Sensitive', 'label'] = 0
df_label.loc[df_label['PLATINUM_STATUS'] == 'Resistant', 'label'] = 1

In [145]:
df_label

,PLATINUM_STATUS,label
ID,,
TCGA-61-1910,Sensitive,0.0
TCGA-24-0982,Resistant,1.0
TCGA-36-1580,Resistant,1.0
TCGA-25-1321,Sensitive,0.0
TCGA-30-1892,Resistant,1.0
...,...,...
TCGA-04-1347,Sensitive,0.0
TCGA-24-0968,Sensitive,0.0
TCGA-30-1891,Resistant,1.0


In [146]:
df_data = pd.read_csv('./Platin_Data/mRNA_tpm_minmax_exp300.csv', sep = '\t', index_col = 0)

In [147]:
df_data

,TCGA-61-1910,TCGA-24-0982,TCGA-36-1580,TCGA-25-1321,TCGA-30-1892,TCGA-61-1741,TCGA-13-0727,TCGA-04-1542,TCGA-61-1918,TCGA-25-1316,...,TCGA-04-1530,TCGA-13-0897,TCGA-09-1666,TCGA-25-2398,TCGA-20-1682,TCGA-04-1347,TCGA-24-0968,TCGA-30-1891,TCGA-09-0366,TCGA-13-0884
ID,,,,,,,,,,,,,,,,,,,,,
ENSG00000131096,0.000117,0.000006,0.000009,0.000012,0.000004,4.750000e-07,0.000005,0.000081,0.000136,0.000171,...,0.002422,0.000048,0.000097,0.000004,6.198005e-07,6.562605e-07,0.000004,1.548384e-07,0.000038,0.000018
ENSG00000187581,0.003695,0.000046,0.000162,0.092906,0.000079,7.387742e-06,0.000000,0.003800,0.002140,0.000250,...,0.000000,0.000101,0.000000,0.000038,1.365541e-05,1.927826e-05,0.000073,6.822780e-06,0.001401,0.000000
ENSG00000047936,0.000511,0.000002,0.000003,0.000149,0.000008,2.522107e-07,0.000005,0.000034,0.000245,0.000002,...,0.000009,0.000016,0.000250,0.000002,1.677093e-04,4.113386e-07,0.000007,5.823084e-08,0.000045,0.000041
ENSG00000276775,0.000000,0.000700,0.000382,0.000000,0.001111,5.486066e-04,0.000000,0.000000,0.000000,0.001771,...,0.000175,0.001909,0.003346,0.001787,8.627366e-03,0.000000e+00,0.001378,2.251791e-04,0.000220,0.000000
ENSG00000186198,0.000217,0.000008,0.000115,0.002635,0.000046,1.082798e-06,0.000031,0.000348,0.001129,0.000477,...,0.000012,0.000430,0.001456,0.000022,1.200858e-05,2.825552e-05,0.000003,9.999929e-06,0.000164,0.000346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000236333,0.000354,0.000010,0.000055,0.000203,0.000712,9.533354e-06,0.000003,0.000041,0.000074,0.000120,...,0.000097,0.000004,0.000021,0.000159,2.969887e-06,7.547000e-06,0.000015,0.000000e+00,0.000000,0.000011
ENSG00000138435,0.000430,0.000012,0.000068,0.000040,0.000073,6.450000e-07,0.000023,0.000124,0.000199,0.000539,...,0.000110,0.001808,0.000096,0.000412,1.589515e-05,1.009811e-05,0.000015,7.941842e-06,0.000033,0.000072
ENSG00000211639,0.000000,0.000195,0.036216,0.000334,0.000000,2.999000e-06,0.000145,0.000000,0.007296,0.000000,...,0.000261,0.000493,0.000000,0.000000,9.978080e-04,4.155584e-03,0.000667,9.970884e-05,0.000683,0.000907


In [148]:
df_data_mod = df_data.T

In [149]:
df_data_mod

ID,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000276775,ENSG00000186198,ENSG00000280411,ENSG00000186897,ENSG00000179914,ENSG00000138136,ENSG00000233048,...,ENSG00000258170,ENSG00000178796,ENSG00000200087,ENSG00000251546,ENSG00000279834,ENSG00000236333,ENSG00000138435,ENSG00000211639,ENSG00000211717,ENSG00000271130
TCGA-61-1910,1.173890e-04,0.003695,5.108446e-04,0.000000,0.000217,0.003498,0.009984,0.000100,0.000000,0.000000e+00,...,0.000000,0.000000,1.000000,0.000000,0.004107,0.000354,0.000430,0.000000,0.124171,0.000000
TCGA-24-0982,6.481502e-06,0.000046,2.437531e-06,0.000700,0.000008,0.000054,0.000051,0.000065,0.000000,0.000000e+00,...,0.000002,0.000006,0.252776,0.000000,0.000005,0.000010,0.000012,0.000195,0.000000,0.000060
TCGA-36-1580,9.202889e-06,0.000162,2.999513e-06,0.000382,0.000115,0.006847,0.000943,0.000004,0.000009,0.000000e+00,...,0.000006,0.000010,0.686048,0.000572,0.000000,0.000055,0.000068,0.036216,0.000491,0.000000
TCGA-25-1321,1.245761e-05,0.092906,1.487486e-04,0.000000,0.002635,0.004223,0.007855,0.000093,0.000000,5.535386e-06,...,0.000010,0.002686,0.089683,0.001614,0.000445,0.000203,0.000040,0.000334,0.002493,0.000000
TCGA-30-1892,4.456219e-06,0.000079,8.379363e-06,0.001111,0.000046,0.000744,0.000125,0.000000,0.000000,0.000000e+00,...,0.000036,0.000012,1.000000,0.001201,0.000000,0.000712,0.000073,0.000000,0.000000,0.000207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,6.562605e-07,0.000019,4.113386e-07,0.000000,0.000028,0.000730,0.005411,0.000009,0.000003,0.000000e+00,...,0.000000,0.000087,0.002397,0.000227,0.000000,0.000008,0.000010,0.004156,0.000657,0.000000
TCGA-24-0968,4.146222e-06,0.000073,6.860879e-06,0.001378,0.000003,0.001946,0.004744,0.001296,0.001681,2.456431e-07,...,0.000187,0.000009,0.365370,0.001075,0.000000,0.000015,0.000015,0.000667,0.000041,0.000145
TCGA-30-1891,1.548384e-07,0.000007,5.823084e-08,0.000225,0.000010,0.006880,0.000059,0.000003,0.000002,4.586703e-07,...,0.000000,0.000001,0.095015,0.001782,0.000000,0.000000,0.000008,0.000100,0.000031,0.000090
TCGA-09-0366,3.815800e-05,0.001401,4.464524e-05,0.000220,0.000164,0.001327,0.001446,0.001663,0.000000,6.279640e-06,...,0.000040,0.000368,0.019988,0.000440,0.000130,0.000000,0.000033,0.000683,0.000212,0.000000


In [150]:
#df_label_mi = pd.read_csv('./Platin_Data/miRNA_patient.csv', sep = '\t', index_col = 0)

In [151]:
df_data_mi = pd.read_csv('./Platin_Data/miRNA_rpm_minmax_exp100.csv', sep = '\t', index_col = 0)

In [152]:
df_data_mi = df_data_mi.T

In [153]:
#df_mi = pd.concat([df_label_mi,df_data_mi], axis = 1)

In [154]:
df = pd.concat([df_label,df_data_mod,df_data_mi], axis = 1)

In [304]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size = 0.2)

In [305]:
train

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000276775,ENSG00000186198,ENSG00000280411,ENSG00000186897,ENSG00000179914,...,hsa-mir-101-2,hsa-mir-514a-1,hsa-mir-6884,hsa-mir-514a-2,hsa-mir-331,hsa-mir-550a-3,hsa-mir-6751,hsa-mir-6757,hsa-mir-3620,hsa-mir-1224
ID,,,,,,,,,,,,,,,,,,,,,
TCGA-25-1318,Resistant,1.0,1.596624e-05,0.000000e+00,1.020765e-04,0.000000,0.002269,0.013322,0.059280,0.000287,...,0.007110,0.000235,0.000450,0.001418,0.147297,0.000880,0.001466,0.000000,0.012875,0.000124
TCGA-25-1628,Resistant,1.0,7.747982e-07,3.414062e-05,1.165529e-06,0.000225,0.000008,0.005406,0.000411,0.000003,...,0.002814,0.000092,0.000640,0.000246,0.007597,0.000040,0.000000,0.000000,0.000042,0.000024
TCGA-29-2414,Sensitive,0.0,6.979534e-06,1.153295e-04,2.952932e-06,0.001994,0.000096,0.005278,0.007007,0.000172,...,0.003853,0.000000,0.000000,0.000000,0.006267,0.000129,0.000000,0.000000,0.000053,0.000027
TCGA-23-1026,Sensitive,0.0,8.569000e-06,2.157631e-04,9.207438e-06,0.000381,0.000103,0.000511,0.000649,0.000037,...,0.019660,0.000811,0.000205,0.000069,0.013577,0.000412,0.000410,0.000025,0.002295,0.001501
TCGA-13-0762,Sensitive,0.0,5.452678e-05,1.554663e-03,2.412489e-06,0.000333,0.000104,0.001338,0.069856,0.045322,...,0.000135,0.000004,0.000000,0.000000,0.003941,0.000505,0.000510,0.000014,0.000236,0.000070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-25-2400,Sensitive,0.0,1.556551e-04,3.609877e-03,1.540473e-05,0.004255,0.000212,0.005127,0.030949,0.000540,...,0.006285,0.000837,0.000002,0.022045,0.059664,0.001082,0.000136,0.000007,0.000796,0.000382
TCGA-30-1891,Resistant,1.0,1.548384e-07,6.822780e-06,5.823084e-08,0.000225,0.000010,0.006880,0.000059,0.000003,...,0.006675,0.000000,0.000502,0.000013,0.005920,0.000380,0.000058,0.000000,0.001103,0.001839
TCGA-24-1565,Resistant,1.0,3.730177e-05,6.574643e-04,3.460305e-05,0.001292,0.003710,0.001037,0.002961,0.000119,...,0.001431,0.012438,0.000168,0.023048,0.015474,0.000064,0.000000,0.000000,0.000000,0.000000


In [306]:
train.to_csv("train_platin_total.csv", mode='w')

In [307]:
val

,PLATINUM_STATUS,label,ENSG00000131096,ENSG00000187581,ENSG00000047936,ENSG00000276775,ENSG00000186198,ENSG00000280411,ENSG00000186897,ENSG00000179914,...,hsa-mir-101-2,hsa-mir-514a-1,hsa-mir-6884,hsa-mir-514a-2,hsa-mir-331,hsa-mir-550a-3,hsa-mir-6751,hsa-mir-6757,hsa-mir-3620,hsa-mir-1224
ID,,,,,,,,,,,,,,,,,,,,,
TCGA-25-1323,Sensitive,0.0,3.819175e-04,0.380862,4.233294e-04,0.000000,0.001947,0.000000,0.022411,4.212317e-03,...,0.000892,0.003095,0.000008,0.004870,0.002314,0.000791,0.000080,0.000000,2.840923e-04,0.001293
TCGA-23-1028,Resistant,1.0,2.955671e-06,0.000037,1.429141e-06,0.000088,0.000022,0.004051,0.001261,4.297779e-05,...,0.026673,0.000000,0.000658,0.000611,0.003870,0.000509,0.000000,0.000000,1.165933e-04,0.000973
TCGA-09-0364,Sensitive,0.0,4.683994e-03,0.008449,1.030136e-05,0.000000,0.001061,0.002857,0.297989,2.320672e-01,...,0.002537,0.009067,0.000224,0.003903,0.000400,0.000363,0.000052,0.000044,8.353465e-04,0.001583
TCGA-61-1741,Sensitive,0.0,4.750000e-07,0.000007,2.522107e-07,0.000549,0.000001,0.010903,0.000186,1.960841e-04,...,0.000384,0.000174,0.000000,0.000009,0.000083,0.000015,0.000000,0.000000,3.717121e-06,0.000076
TCGA-24-1930,Sensitive,0.0,2.747104e-07,0.000007,2.988093e-06,0.018540,0.000002,0.023410,0.000006,5.060912e-07,...,0.002293,0.000000,0.000005,0.000023,0.010144,0.000502,0.000000,0.000013,1.241265e-04,0.000000
TCGA-10-0938,Resistant,1.0,6.062062e-05,0.000000,1.393205e-05,0.000000,0.020380,0.000000,0.002078,4.436165e-04,...,0.001976,0.000147,0.000013,0.000227,0.032827,0.000127,0.000000,0.000000,3.919542e-04,0.000309
TCGA-29-1693,Sensitive,0.0,5.582000e-06,0.002091,2.431982e-03,0.000580,0.000216,0.004657,0.000423,2.172403e-04,...,0.005006,0.001083,0.000026,0.000000,0.001026,0.002186,0.000017,0.000000,8.808227e-04,0.001141
TCGA-13-0901,Sensitive,0.0,1.437657e-05,0.000745,3.816472e-06,0.000264,0.000087,0.000265,0.000650,2.784878e-04,...,0.001317,0.000226,0.000008,0.000000,0.076949,0.002623,0.000081,0.000000,1.119844e-04,0.002365
TCGA-61-1911,Sensitive,0.0,1.003716e-07,0.000247,7.171980e-07,0.000067,0.000000,0.034280,0.000004,4.807708e-07,...,0.005696,0.000004,0.000000,0.000801,0.001186,0.000101,0.000102,0.000580,4.556330e-04,0.013226


In [308]:
val.to_csv("val_platin_total.csv", mode='w')

In [309]:
train = pd.read_csv("train_platin_total.csv")
val = pd.read_csv("val_platin_total.csv")

In [310]:
trn_X_pd = train.drop(["ID","PLATINUM_STATUS","label"],axis=1).values
trn_y_pd = train.label.values

print("Information of Training Data")
print("Samples : {}".format(trn_X_pd.shape[0]))
print("Features : {}".format(trn_X_pd.shape[1]))

Information of Training Data
Samples : 169
Features : 400


In [311]:
val_X_pd = val.drop(["ID","PLATINUM_STATUS","label"],axis=1).values
val_y_pd = val.label.values

print("Information of Training Data")
print("Samples : {}".format(val_X_pd.shape[0]))
print("Features : {}".format(val_X_pd.shape[1]))

Information of Training Data
Samples : 43
Features : 400


In [312]:
item = []
item_2 = []

print( (val_y_pd == 1.0).sum())
print( (val_y_pd == 0.0).sum())

print( (trn_y_pd == 1.0).sum())
print( (trn_y_pd == 0.0).sum())

10
33
51
118


In [313]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5 # Number of K-fold Splits

splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True).split(trn_X_pd, trn_y_pd))
splits[:5]

[(array([  0,   1,   2,   3,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  27,  28,
          29,  31,  33,  34,  35,  36,  37,  39,  40,  41,  42,  43,  44,
          47,  48,  49,  50,  51,  52,  53,  54,  56,  58,  59,  60,  61,
          62,  63,  64,  65,  66,  68,  70,  71,  72,  73,  76,  77,  78,
          79,  80,  81,  82,  85,  86,  87,  88,  89,  90,  91,  93,  94,
          96,  97,  98, 100, 101, 102, 103, 104, 105, 107, 108, 111, 112,
         113, 114, 118, 119, 121, 124, 125, 126, 127, 128, 129, 130, 131,
         132, 134, 135, 137, 138, 139, 140, 141, 142, 143, 144, 146, 148,
         149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162,
         163, 165, 166, 167]),
  array([  4,  25,  26,  30,  32,  38,  45,  46,  55,  57,  67,  69,  74,
          75,  83,  84,  92,  95,  99, 106, 109, 110, 115, 116, 117, 120,
         122, 123, 133, 136, 145, 147, 159, 164, 168])),
 (array([  0,   3,   4, 

In [314]:
for i, (train_idx, valid_idx) in enumerate(splits):

    print( (trn_y_pd[valid_idx.astype(int)] == 1.0).sum())
    print( (trn_y_pd[valid_idx.astype(int)] == 0.0).sum())
    
    #print( (trn_y_pd[train_idx.astype(int)] == 1.0).sum())
    #print( (trn_y_pd[train_idx.astype(int)] == 0.0).sum())

11
24
10
24
10
24
10
23
10
23


In [315]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [316]:
batch_size = 5

In [317]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [318]:
model = DNN_seq()

In [319]:
class CyclicLR(object):
    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [320]:
from torch.optim.optimizer import Optimizer

import time # ??

from sklearn.metrics import roc_auc_score

In [321]:
print(trn_X_pd.shape[0])
          
train_preds = np.zeros(trn_X_pd.shape[0])
test_preds = np.zeros(val_X_pd.shape[0])


train_target = train.label

169


In [322]:
avg_losses_f = []
avg_val_losses_f = []

In [336]:
for i, (train_idx, valid_idx) in enumerate(splits):
    
    ## ???
    x_train_fold = torch.tensor(trn_X_pd[train_idx.astype(int)], dtype=torch.float)   # use_cuse?
    y_train_fold = torch.tensor(trn_y_pd[train_idx.astype(int), np.newaxis], dtype=torch.float32)    

    x_val_fold = torch.tensor(trn_X_pd[valid_idx.astype(int)], dtype=torch.float)
    y_val_fold = torch.tensor(trn_y_pd[valid_idx.astype(int), np.newaxis], dtype=torch.float32)  
    
################################################################################    
    trn_X = torch.from_numpy(trn_X_pd[train_idx.astype(int)].astype(float))
    trn_y = torch.from_numpy(trn_y_pd[train_idx.astype(int)].astype(float))
    
    # Train
    trn = Dataset(trn_X, trn_y)
    trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)
    trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)
    
    # Valid
    valid_X = torch.from_numpy(trn_X_pd[valid_idx.astype(int)].astype(float))
    valid_y = torch.from_numpy(trn_y_pd[valid_idx.astype(int)].astype(float))
    
    valid = Dataset(valid_X, valid_y)
    valid_loader = data_utils.DataLoader(valid, batch_size=batch_size, shuffle=False, drop_last = True)
    
    # Test
    val_X = torch.from_numpy(val_X_pd.astype(float))
    val_y = torch.from_numpy(val_y_pd.astype(float))
    val = Dataset(val_X, val_y)
    test_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)
   ################################################################################    
    print(i)
    
    ## Loss function
    criterion = nn.BCELoss()
    learning_rate = 1e-3
    
    ## After just second, will modify optimizer(weight_decay,step_size, base_lr, max_lr,scheduler)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 1e-5)
    
    model = DNN_seq()
    use_cuda = torch.cuda.is_available()
    num_epochs = 100
    
    if use_cuda:
        model = model.cuda()
    
    ##############################################################################
    step_size = 2000
    base_lr, max_lr = 0.001, 0.005  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                             lr=max_lr)
    
    scheduler = CyclicLR(optimizer, base_lr=base_lr, max_lr=max_lr, step_size=step_size, mode='exp_range', gamma=0.99994)    
    
    ##############################################################################
    
    print(f'Fold {i+1}')
    
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        avg_loss = 0.
        #correct = 0.   # Accuracy
        
        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)
            if scheduler:
                #print('cycle_LR')
                scheduler.batch_step()
            #print(trn_pred.squeeze())
            #print(trn_y)
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            trn_loss.backward()
            optimizer.step()
        
            avg_loss += trn_loss.item()/len(trn_loader)
        
            #predicted = torch.max(trn_pred.data, 1)[1] 
            #correct += (predicted == trn_y).sum()
        model.eval()
        
        valid_preds_fold = np.zeros((valid_X.size(0)))
        test_preds_fold = np.zeros(val_X_pd.shape[0])  # Test
        
        avg_val_loss = 0.
        
        for batch_idx, val in enumerate(valid_loader):
            val_X, val_y = val['X'], val['y']
            if use_cuda:
                val_X, val_y = val_X.cuda(), val_y.cuda()
            val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
            optimizer.zero_grad()
            val_pred = model(val_X).detach()
            val_loss = criterion(val_pred.squeeze(), val_y)
        
            avg_val_loss += val_loss.item()/len(val_loader)
            valid_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (val_pred.cpu().numpy())[:, 0]
            # Loss function chage -> plus Sigmoid
        elapsed_time = time.time() - start_time 
        print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t time={:.2f}s'.format(
            epoch + 1, num_epochs, avg_loss, avg_val_loss, elapsed_time))            

# Test        
    avg_losses_f.append(avg_loss)
    avg_val_losses_f.append(avg_val_loss) 
    
    for batch_idx, test in enumerate(test_loader):
        test_X, test_y = test['X'], test['y']
        if use_cuda:
            test_X, test_y = test_X.cuda(), test_y.cuda()
        test_X, test_y = Variable(test_X).float(), Variable(test_y).float()        
        test_pred = model(test_X).detach()

        test_preds_fold[batch_idx * batch_size:(batch_idx+1) * batch_size] = (test_pred.cpu().numpy())[:, 0]
        
    train_preds[valid_idx] = valid_preds_fold
    test_preds += test_preds_fold / len(splits)

auc  =  round(roc_auc_score(train_target,train_preds),4)      
print('All \t loss={:.4f} \t val_loss={:.4f} \t auc={:.4f}'.format(np.average(avg_losses_f),np.average(avg_val_losses_f),auc))        
        

0
Fold 1
Epoch 1/100 	 loss=0.6784 	 val_loss=0.6360 	 time=0.29s
Epoch 2/100 	 loss=0.6340 	 val_loss=0.6458 	 time=0.24s
Epoch 3/100 	 loss=0.5667 	 val_loss=0.6749 	 time=0.25s
Epoch 4/100 	 loss=0.5258 	 val_loss=0.6464 	 time=0.26s
Epoch 5/100 	 loss=0.5714 	 val_loss=0.7135 	 time=0.26s
Epoch 6/100 	 loss=0.5852 	 val_loss=0.6816 	 time=0.25s
Epoch 7/100 	 loss=0.5204 	 val_loss=0.6829 	 time=0.25s
Epoch 8/100 	 loss=0.4294 	 val_loss=0.6950 	 time=0.26s
Epoch 9/100 	 loss=0.5285 	 val_loss=0.6044 	 time=0.26s
Epoch 10/100 	 loss=0.4744 	 val_loss=0.6351 	 time=0.26s
Epoch 11/100 	 loss=0.5075 	 val_loss=0.6434 	 time=0.26s
Epoch 12/100 	 loss=0.4881 	 val_loss=0.7161 	 time=0.26s
Epoch 13/100 	 loss=0.4262 	 val_loss=0.6587 	 time=0.26s
Epoch 14/100 	 loss=0.3817 	 val_loss=0.7462 	 time=0.26s
Epoch 15/100 	 loss=0.4540 	 val_loss=0.7404 	 time=0.26s
Epoch 16/100 	 loss=0.4281 	 val_loss=0.6665 	 time=0.29s
Epoch 17/100 	 loss=0.4790 	 val_loss=0.6660 	 time=0.28s
Epoch 18/100 	

Epoch 43/100 	 loss=0.2224 	 val_loss=0.8427 	 time=0.28s
Epoch 44/100 	 loss=0.3896 	 val_loss=0.7022 	 time=0.28s
Epoch 45/100 	 loss=0.2874 	 val_loss=0.7448 	 time=0.28s
Epoch 46/100 	 loss=0.2693 	 val_loss=0.7266 	 time=0.28s
Epoch 47/100 	 loss=0.3082 	 val_loss=0.7996 	 time=0.28s
Epoch 48/100 	 loss=0.3126 	 val_loss=0.7363 	 time=0.28s
Epoch 49/100 	 loss=0.2847 	 val_loss=0.7401 	 time=0.28s
Epoch 50/100 	 loss=0.2609 	 val_loss=0.8231 	 time=0.28s
Epoch 51/100 	 loss=0.4314 	 val_loss=0.6395 	 time=0.28s
Epoch 52/100 	 loss=0.3598 	 val_loss=0.6529 	 time=0.28s
Epoch 53/100 	 loss=0.3113 	 val_loss=0.6720 	 time=0.28s
Epoch 54/100 	 loss=0.3380 	 val_loss=0.6188 	 time=0.28s
Epoch 55/100 	 loss=0.3192 	 val_loss=0.5659 	 time=0.28s
Epoch 56/100 	 loss=0.2853 	 val_loss=0.6429 	 time=0.28s
Epoch 57/100 	 loss=0.3579 	 val_loss=0.6490 	 time=0.28s
Epoch 58/100 	 loss=0.3624 	 val_loss=0.6138 	 time=0.28s
Epoch 59/100 	 loss=0.4326 	 val_loss=0.6227 	 time=0.28s
Epoch 60/100 	

Epoch 85/100 	 loss=0.2556 	 val_loss=0.6319 	 time=0.27s
Epoch 86/100 	 loss=0.3635 	 val_loss=0.5732 	 time=0.27s
Epoch 87/100 	 loss=0.2404 	 val_loss=0.6461 	 time=0.26s
Epoch 88/100 	 loss=0.2802 	 val_loss=0.6096 	 time=0.28s
Epoch 89/100 	 loss=0.2791 	 val_loss=0.6619 	 time=0.27s
Epoch 90/100 	 loss=0.3344 	 val_loss=0.6539 	 time=0.27s
Epoch 91/100 	 loss=0.2250 	 val_loss=0.6892 	 time=0.27s
Epoch 92/100 	 loss=0.2502 	 val_loss=0.7360 	 time=0.27s
Epoch 93/100 	 loss=0.2157 	 val_loss=0.6850 	 time=0.28s
Epoch 94/100 	 loss=0.3403 	 val_loss=0.7311 	 time=0.27s
Epoch 95/100 	 loss=0.2789 	 val_loss=0.5420 	 time=0.28s
Epoch 96/100 	 loss=0.2748 	 val_loss=0.5592 	 time=0.27s
Epoch 97/100 	 loss=0.2493 	 val_loss=0.5132 	 time=0.27s
Epoch 98/100 	 loss=0.2939 	 val_loss=0.5141 	 time=0.27s
Epoch 99/100 	 loss=0.2978 	 val_loss=0.5656 	 time=0.27s
Epoch 100/100 	 loss=0.1846 	 val_loss=0.6038 	 time=0.28s
3
Fold 4
Epoch 1/100 	 loss=0.6479 	 val_loss=0.6198 	 time=0.27s
Epoch

Epoch 27/100 	 loss=0.3767 	 val_loss=0.5005 	 time=0.29s
Epoch 28/100 	 loss=0.3505 	 val_loss=0.6167 	 time=0.31s
Epoch 29/100 	 loss=0.4578 	 val_loss=0.6372 	 time=0.29s
Epoch 30/100 	 loss=0.3651 	 val_loss=0.5680 	 time=0.28s
Epoch 31/100 	 loss=0.4103 	 val_loss=0.6478 	 time=0.29s
Epoch 32/100 	 loss=0.4017 	 val_loss=0.5742 	 time=0.29s
Epoch 33/100 	 loss=0.3411 	 val_loss=0.5443 	 time=0.28s
Epoch 34/100 	 loss=0.4023 	 val_loss=0.6041 	 time=0.28s
Epoch 35/100 	 loss=0.4121 	 val_loss=0.5228 	 time=0.28s
Epoch 36/100 	 loss=0.4279 	 val_loss=0.6575 	 time=0.28s
Epoch 37/100 	 loss=0.4115 	 val_loss=0.6027 	 time=0.27s
Epoch 38/100 	 loss=0.3367 	 val_loss=0.5639 	 time=0.27s
Epoch 39/100 	 loss=0.3768 	 val_loss=0.5816 	 time=0.27s
Epoch 40/100 	 loss=0.2796 	 val_loss=0.5095 	 time=0.27s
Epoch 41/100 	 loss=0.3478 	 val_loss=0.5716 	 time=0.27s
Epoch 42/100 	 loss=0.3081 	 val_loss=0.5851 	 time=0.27s
Epoch 43/100 	 loss=0.2916 	 val_loss=0.7197 	 time=0.29s
Epoch 44/100 	

In [337]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Train', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_loader)*batch_size)))
print('*'*46)
print("Train accuracy:{:.3f}".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.07715117186307907, Accuracy: 0.7037037014961243 %
**********************************************
Train accuracy:0.704


In [338]:

model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())

        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(test_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(test_loader)*batch_size)))

******************** Test ********************
Loss: 0.5557433366775513, Accuracy: 0.7333333492279053 %
**********************************************
Val accuracy:0.733


In [330]:
trn_X_pd

array([[1.59662362e-05, 0.00000000e+00, 1.02076500e-04, ...,
        0.00000000e+00, 1.28752303e-02, 1.23934200e-04],
       [7.74798159e-07, 3.41406163e-05, 1.16552895e-06, ...,
        0.00000000e+00, 4.21414921e-05, 2.42437585e-05],
       [6.97953402e-06, 1.15329500e-04, 2.95293198e-06, ...,
        0.00000000e+00, 5.33686870e-05, 2.74985439e-05],
       ...,
       [3.73017687e-05, 6.57464300e-04, 3.46030501e-05, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.34393991e-05, 5.16415500e-04, 2.42411746e-05, ...,
        0.00000000e+00, 9.87277592e-06, 1.45977700e-04],
       [6.58817034e-08, 9.67667916e-07, 2.47764632e-08, ...,
        0.00000000e+00, 1.33158250e-03, 8.23836230e-03]])

In [215]:
trn_X = torch.from_numpy(trn_X_pd.astype(float))
trn_y = torch.from_numpy(trn_y_pd.astype(float))

val_X = torch.from_numpy(val_X_pd.astype(float))
val_y = torch.from_numpy(val_y_pd.astype(float))

In [216]:
batch_size = 5

In [217]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)


In [218]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True, drop_last = True)

trn_test_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=False)


val = Dataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

In [219]:
tmp = next(iter(trn_loader))

In [220]:
tmp

{'X': tensor([[6.5677e-05, 8.3347e-02, 3.8531e-05,  ..., 1.9207e-03, 3.3437e-04,
          4.4619e-04],
         [1.9899e-05, 2.3116e-03, 0.0000e+00,  ..., 0.0000e+00, 9.3073e-05,
          1.6355e-04],
         [4.5190e-07, 1.4934e-05, 3.8238e-07,  ..., 0.0000e+00, 5.9395e-04,
          3.6635e-03],
         [3.8192e-04, 3.8086e-01, 4.2333e-04,  ..., 0.0000e+00, 2.8409e-04,
          1.2932e-03],
         [1.9966e-05, 7.4559e-06, 1.2918e-05,  ..., 0.0000e+00, 6.3502e-04,
          1.8207e-02]], dtype=torch.float64),
 'y': tensor([0., 1., 0., 0., 1.], dtype=torch.float64)}

In [221]:
num_batches = len(trn_loader)

## Build Model

In [222]:
use_cuda = torch.cuda.is_available()

In [223]:
class MLPRegressor(nn.Module):
    
    def __init__(self):
        super(MLPRegressor, self).__init__()
        h1 = nn.Linear((len(train.columns)-3), 32)
        h2 = nn.Linear(32, 16)
        h3 = nn.Linear(16, 1)
        self.hidden = nn.Sequential(
            h1,
            nn.ReLU(),
            h2,
            nn.ReLU(),
            h3,
        )
        if use_cuda:
            self.hidden = self.hidden.cuda()
        
    def forward(self, x):
        o = self.hidden(x)
        return o.view(-1)

In [224]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
         
        self.layer1 = nn.Sequential(
            torch.nn.Linear((len(train.columns)-3), 256, bias=True),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU()
        )
 
        self.layer2 = nn.Sequential(
            torch.nn.Linear(256, 128, bias=True),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU()
        )
    
        self.layer3 = nn.Sequential(
            torch.nn.Linear(128, 64, bias=True),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU()
        )
 
        self.layer4 = nn.Sequential(
            torch.nn.Linear(64, 1, bias=True),
            
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.layer1(x)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer2(x_out)
        x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer3(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        x_out = self.layer4(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [332]:
class DNN_seq_(nn.Module):
    def __init__(self):
        super(DNN_seq_, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
        )
        self.classifier2 = nn.Sequential(

            torch.nn.Linear(100, 100, bias=True),
            torch.nn.BatchNorm1d(100),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            

            #torch.nn.Dropout(0.3)
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(100,1, bias=True)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [259]:
#[150,200,100,50]
#[150,200,100,50,20]

class DNN_seq(nn.Module):
    def __init__(self):
        super(DNN_seq, self).__init__()
        
        
        self.classifier = nn.Sequential(
            torch.nn.Linear((trn_X_pd.shape[1]), 150, bias=True),
            torch.nn.BatchNorm1d(150),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.classifier2 = nn.Sequential(    
            torch.nn.Linear(150, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
        )
        
        self.classifier3 = nn.Sequential(  
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            torch.nn.Linear(250, 250, bias=True),
            torch.nn.BatchNorm1d(250),
            torch.nn.ReLU(),
            nn.Dropout(0.3),
            
            
        )
        self.output_layer = nn.Sequential(
            torch.nn.Linear(250, 1, bias=True),
            #nn.Dropout(0.3)
        )
    
    def forward(self, x):
        x = x.view(x.size(0), -1) # flatten
        x_out = self.classifier(x)
        x_out = self.classifier2(x_out)
        x_out = self.classifier3(x_out)
        x_out = self.output_layer(x_out)
        #x_out = F.dropout(x_out, p=0.5, training=self.training)
        return torch.sigmoid(x_out)
 


In [245]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight) 

In [246]:
model = DNN_seq()

In [247]:
model.apply(weights_init)

DNN_seq(
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=150, bias=True)
    (1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier2): Sequential(
    (0): Linear(in_features=150, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
  )
  (classifier3): Sequential(
    (0): Linear(in_features=250, out_features=250, bias=True)
    (1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=250, out_features=250, bias=True)
    (5): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
  )
  (output_layer): Sequential(
    (0): Linear(in_features=250, ou

## Train model

In [248]:
criterion = nn.BCELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [249]:
num_epochs = 20
num_batches = len(trn_loader) # requier modification

In [250]:
num_batches

36

In [251]:
trn_X_pd.shape[1]

400

# Train_2

In [252]:
model = DNN_seq()

In [253]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)#,lr=0.001, betas=(0.9,0.999))
    error = nn.BCELoss()
    num_epochs = 20
    
    #checkpoint = torch.load("./platin_model_save/First_platin_model.pth")                                                                                  
    #model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
    #num_epochs = checkpoint['epoch']                                                                                                           
    #loss = checkpoint['loss']        
    
    
    model.train()
    for epoch in range(num_epochs):
        trn_loss_summary = 0.0
        correct = 0   # Accuracy

        for batch_idx, trn in enumerate(trn_loader):
            trn_X, trn_y = trn['X'], trn['y']
            if use_cuda:
                trn_X, trn_y = trn_X.cuda(), trn_y.cuda()
            trn_X, trn_y = Variable(trn_X).float(), Variable(trn_y).float()
            optimizer.zero_grad()
            trn_pred = model(trn_X)
            #trn_pred = trn_pred.type()
            #type_as(torch.cuda.IntTensor())
            trn_loss = criterion(trn_pred.squeeze(), trn_y)
            trn_loss.backward()
            optimizer.step()
        
            trn_loss_summary += trn_loss
        
            predicted = torch.max(trn_pred.data, 1)[1] 
            correct += (predicted == trn_y).sum()
            #print(correct)
            if batch_idx % 2 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(trn_X), len(trn_loader.dataset), 100.*batch_idx / len(trn_loader), trn_loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))
    torch.save({
    #'epoch': EPOCHS,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': trn_loss
        }, "./platin_model_save/platin_model_2.pth")    

In [254]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/180 (0%)]	Loss: 0.776929	 Accuracy:80.000%
Epoch : 0 [10/180 (6%)]	Loss: 0.801617	 Accuracy:80.000%
Epoch : 0 [20/180 (11%)]	Loss: 0.696659	 Accuracy:76.000%
Epoch : 0 [30/180 (17%)]	Loss: 0.769450	 Accuracy:74.286%
Epoch : 0 [40/180 (22%)]	Loss: 0.563943	 Accuracy:75.556%
Epoch : 0 [50/180 (28%)]	Loss: 0.615975	 Accuracy:76.364%
Epoch : 0 [60/180 (33%)]	Loss: 0.621697	 Accuracy:72.308%
Epoch : 0 [70/180 (39%)]	Loss: 0.754056	 Accuracy:68.000%
Epoch : 0 [80/180 (44%)]	Loss: 0.327390	 Accuracy:68.235%
Epoch : 0 [90/180 (50%)]	Loss: 0.497569	 Accuracy:69.474%
Epoch : 0 [100/180 (56%)]	Loss: 0.407298	 Accuracy:72.381%
Epoch : 0 [110/180 (61%)]	Loss: 0.496243	 Accuracy:72.174%
Epoch : 0 [120/180 (67%)]	Loss: 0.654800	 Accuracy:71.200%
Epoch : 0 [130/180 (72%)]	Loss: 0.469441	 Accuracy:70.370%
Epoch : 0 [140/180 (78%)]	Loss: 0.593599	 Accuracy:71.034%
Epoch : 0 [150/180 (83%)]	Loss: 0.512662	 Accuracy:71.613%
Epoch : 0 [160/180 (89%)]	Loss: 0.658217	 Accuracy:70.303%
Epoch : 0 

Epoch : 8 [30/180 (17%)]	Loss: 0.306748	 Accuracy:68.571%
Epoch : 8 [40/180 (22%)]	Loss: 0.185420	 Accuracy:71.111%
Epoch : 8 [50/180 (28%)]	Loss: 0.471007	 Accuracy:72.727%
Epoch : 8 [60/180 (33%)]	Loss: 0.256661	 Accuracy:72.308%
Epoch : 8 [70/180 (39%)]	Loss: 0.274515	 Accuracy:74.667%
Epoch : 8 [80/180 (44%)]	Loss: 0.561807	 Accuracy:74.118%
Epoch : 8 [90/180 (50%)]	Loss: 0.323444	 Accuracy:71.579%
Epoch : 8 [100/180 (56%)]	Loss: 0.683152	 Accuracy:71.429%
Epoch : 8 [110/180 (61%)]	Loss: 0.569845	 Accuracy:71.304%
Epoch : 8 [120/180 (67%)]	Loss: 0.381507	 Accuracy:72.000%
Epoch : 8 [130/180 (72%)]	Loss: 0.238484	 Accuracy:72.593%
Epoch : 8 [140/180 (78%)]	Loss: 0.253200	 Accuracy:72.414%
Epoch : 8 [150/180 (83%)]	Loss: 0.248830	 Accuracy:71.613%
Epoch : 8 [160/180 (89%)]	Loss: 0.293989	 Accuracy:71.515%
Epoch : 8 [170/180 (94%)]	Loss: 0.649500	 Accuracy:71.429%
Epoch : 9 [0/180 (0%)]	Loss: 0.588557	 Accuracy:40.000%
Epoch : 9 [10/180 (6%)]	Loss: 0.303254	 Accuracy:66.667%
Epoch : 9

Epoch : 16 [40/180 (22%)]	Loss: 2.041943	 Accuracy:57.778%
Epoch : 16 [50/180 (28%)]	Loss: 0.523941	 Accuracy:65.455%
Epoch : 16 [60/180 (33%)]	Loss: 0.372615	 Accuracy:63.077%
Epoch : 16 [70/180 (39%)]	Loss: 0.314646	 Accuracy:64.000%
Epoch : 16 [80/180 (44%)]	Loss: 0.347928	 Accuracy:68.235%
Epoch : 16 [90/180 (50%)]	Loss: 0.128908	 Accuracy:70.526%
Epoch : 16 [100/180 (56%)]	Loss: 0.631151	 Accuracy:69.524%
Epoch : 16 [110/180 (61%)]	Loss: 0.184766	 Accuracy:68.696%
Epoch : 16 [120/180 (67%)]	Loss: 0.259879	 Accuracy:68.800%
Epoch : 16 [130/180 (72%)]	Loss: 0.405108	 Accuracy:69.630%
Epoch : 16 [140/180 (78%)]	Loss: 0.653986	 Accuracy:71.034%
Epoch : 16 [150/180 (83%)]	Loss: 0.166828	 Accuracy:71.613%
Epoch : 16 [160/180 (89%)]	Loss: 0.140924	 Accuracy:71.515%
Epoch : 16 [170/180 (94%)]	Loss: 0.225211	 Accuracy:71.429%
Epoch : 17 [0/180 (0%)]	Loss: 0.425675	 Accuracy:60.000%
Epoch : 17 [10/180 (6%)]	Loss: 0.384020	 Accuracy:66.667%
Epoch : 17 [20/180 (11%)]	Loss: 0.150301	 Accuracy:

In [255]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(trn_test_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Train', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(trn_loader)*batch_size)))
print('*'*46)
print("Train accuracy:{:.3f}".format(float(correct_val) / (len(trn_loader)*batch_size)))

******************** Train ********************
Loss: 0.4637147784233093, Accuracy: 0.7166666388511658 %
**********************************************
Train accuracy:0.717


In [256]:

model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())

        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 1.1907775402069092, Accuracy: 0.6285714507102966 %
**********************************************
Val accuracy:0.629


In [243]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.396110,0.0
0.776959,0.0
0.542691,0.0
0.377997,0.0
0.365426,0.0
0.380689,1.0
0.145689,1.0
0.157259,0.0
0.366938,0.0


In [260]:
model = DNN_seq()

In [261]:
checkpoint = torch.load("./platin_model_save/First_platin_model.pth")                                                                                  
model.load_state_dict(checkpoint['model_state_dict'])                                                                                         
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])                                                                               
#num_epochs = checkpoint['epoch']                                                                                                           
loss = checkpoint['loss']  

In [262]:
use_cuda = torch.cuda.is_available()

In [263]:

model.eval()
#predict = []
#y_val = val_y
predict = []

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        for i in val_pred:
            predict.append(i.numpy())
        val_pred = val_pred.type_as(torch.FloatTensor())

        val_loss = criterion(val_pred, val_y.unsqueeze(1))
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += torch.sum(predicted == val_y).item()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

RuntimeError: size mismatch, m1: [5 x 400], m2: [500 x 150] at /opt/conda/conda-bld/pytorch_1570710822989/work/aten/src/TH/generic/THTensorMath.cpp:197

In [28]:
val_pred.data

tensor([[0.1228],
        [0.0049]])

In [24]:
predict_ = pd.DataFrame(predict)

predict_.iloc[:,0]

pd.DataFrame(val_y_pd, predict_.iloc[:,0])

,0
0,
0.000213,0.0
0.587282,0.0
0.206532,0.0
0.767732,0.0
0.000056,0.0
0.826447,0.0
0.965310,1.0
0.000252,0.0
0.451954,0.0


# Visualization

In [41]:
plt.figure(figsize=(16,9))
x_range = range(len(trn_loss_list))
plt.plot(x_range, trn_loss_list, label="trn")
plt.plot(x_range, val_loss_list, label="val")
plt.legend()
plt.xlabel("training steps")
plt.ylabel("loss")

NameError: name 'trn_loss_list' is not defined

<Figure size 1152x648 with 0 Axes>

# Transfer Learning

결론적으로 이미 잘 훈련된 모델이 있고, 특히 해당 모델과 유사한 문제를 해결시 transfer learining을 사용합니다.
실질적 조언
새로 훈련할 데이터가 적지만 original 데이터와 유사할 경우

데이터의 양이 적어 fine-tune (전체 모델에 대해서 backpropagation을 진행하는 것) 은 over-fitting의 위험이 있기에 하지 않습니다.
새로 학습할 데이터는 original 데이터와 유사하기 때문에 이 경우 최종 linear classfier 레이어만 학습을 합니다.
새로 훈련할 데이터가 매우 많으며 original 데이터와 유사할 경우

새로 학습할 데이터의 양이 많다는 것은 over-fitting의 위험이 낮다는 뜻이므로, 전체 레이어에 대해서 fine-tune을 합니다.
새로 훈련할 데이터가 적으며 original 데이터와 다른 경우

데이터의 양이 적기 때문에 최종 단계의 linear classifier 레이어를 학습하는 것이 좋을 것입니다. 반면서 데이터가 서로 다르기 때문에 거의 마지막부분 (the top of the network)만 학습하는 것은 좋지 않습니다. 서로 상충이 되는데.. 이 경우에는 네트워크 초기 부분 어딘가 activation 이후에 특정 레이어를 학습시키는게 좋습니다.
새로 훈련할 데이터가 많지만 original 데이터와와 다른 경우

데이터가 많기 때문에 아예 새로운 ConvNet을 만들수도 있지만, 실적적으로 transfer learning이 더 효율이 좋습니다. 전체 네트워크에 대해서 fine-tune을 해도 됩니다.

In [75]:
model.classifier3

Sequential(
  (0): Linear(in_features=32, out_features=16, bias=True)
  (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.3, inplace=False)
)

In [149]:
model_2 = DNN_seq()

In [150]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = model_2.classifier    
model.classifier2 = model_2.classifier2


In [151]:
#model = DNN_seq()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model = model.cuda()
    
fit(model, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.746858	 Accuracy:50.000%
Epoch : 0 [32/171 (18%)]	Loss: 0.662493	 Accuracy:60.417%
Epoch : 0 [64/171 (36%)]	Loss: 0.704260	 Accuracy:60.000%
Epoch : 0 [96/171 (55%)]	Loss: 0.674394	 Accuracy:66.071%
Epoch : 0 [128/171 (73%)]	Loss: 0.722818	 Accuracy:67.361%
Epoch : 0 [110/171 (91%)]	Loss: 0.599336	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.575366	 Accuracy:87.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.667310	 Accuracy:70.833%
Epoch : 1 [64/171 (36%)]	Loss: 0.673750	 Accuracy:68.750%
Epoch : 1 [96/171 (55%)]	Loss: 0.751240	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.740712	 Accuracy:67.361%
Epoch : 1 [110/171 (91%)]	Loss: 0.710288	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.641428	 Accuracy:62.500%
Epoch : 2 [32/171 (18%)]	Loss: 0.618767	 Accuracy:68.750%
Epoch : 2 [64/171 (36%)]	Loss: 0.619703	 Accuracy:65.000%
Epoch : 2 [96/171 (55%)]	Loss: 0.579154	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.667362	 Accuracy:66.667%
Epoch : 2 [110/

In [152]:
model.eval()
#predict = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5934396982192993, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


# Voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV, RidgeClassifier
# VotingClassifier
from sklearn.ensemble import VotingClassifier

# ensemble 할 model 정의
models = [
    ('ada', AdaBoostClassifier()),
    ('bc', BaggingClassifier()),
    ('etc',ExtraTreesClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('rfc', RandomForestClassifier()),
    ('knn', KNeighborsClassifier()),
    ('svc', SVC(probability=True)),
    ('xgb', XGBClassifier()),
    ('lgbm', LGBMClassifier()),
    ('dtc', DecisionTreeClassifier()),
    ('lr', LogisticRegressionCV()),
    ('ridge', RidgeClassifier()),
]

# hard vote
hard_vote  = VotingClassifier(models, voting='hard')
hard_vote_cv = cross_validate(hard_vote, x_train, y_train, cv=k_fold)
hard_vote.fit(x_train, y_train)

# soft vote
soft_vote  = VotingClassifier(models, voting='soft')
soft_vote_cv = cross_validate(soft_vote, x_train, y_train, cv=k_fold)
soft_vote.fit(x_train, y_train)


In [76]:
model_1 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_1 = model_1.cuda()
    
fit(model_1, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.935273	 Accuracy:68.750%
Epoch : 0 [32/171 (18%)]	Loss: 0.791408	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.752347	 Accuracy:68.750%
Epoch : 0 [96/171 (55%)]	Loss: 0.723812	 Accuracy:63.393%
Epoch : 0 [128/171 (73%)]	Loss: 0.811310	 Accuracy:65.972%
Epoch : 0 [110/171 (91%)]	Loss: 0.819974	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.823073	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.663486	 Accuracy:64.583%
Epoch : 1 [64/171 (36%)]	Loss: 0.554099	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.667068	 Accuracy:66.071%
Epoch : 1 [128/171 (73%)]	Loss: 0.624170	 Accuracy:65.278%
Epoch : 1 [110/171 (91%)]	Loss: 0.637554	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.651228	 Accuracy:75.000%
Epoch : 2 [32/171 (18%)]	Loss: 0.534523	 Accuracy:81.250%
Epoch : 2 [64/171 (36%)]	Loss: 0.607727	 Accuracy:71.250%
Epoch : 2 [96/171 (55%)]	Loss: 0.625725	 Accuracy:69.643%
Epoch : 2 [128/171 (73%)]	Loss: 0.518599	 Accuracy:68.750%
Epoch : 2 [110/

In [77]:
model_2 = DNN_seq_()

use_cuda = torch.cuda.is_available()

if use_cuda:
    model_2 = model_2.cuda()
    
fit(model_2, trn_loader)

Epoch : 0 [0/171 (0%)]	Loss: 0.745150	 Accuracy:56.250%
Epoch : 0 [32/171 (18%)]	Loss: 0.593750	 Accuracy:70.833%
Epoch : 0 [64/171 (36%)]	Loss: 0.609681	 Accuracy:72.500%
Epoch : 0 [96/171 (55%)]	Loss: 0.811276	 Accuracy:70.536%
Epoch : 0 [128/171 (73%)]	Loss: 0.909858	 Accuracy:68.750%
Epoch : 0 [110/171 (91%)]	Loss: 0.555978	 Accuracy:65.909%
Epoch : 1 [0/171 (0%)]	Loss: 0.671668	 Accuracy:62.500%
Epoch : 1 [32/171 (18%)]	Loss: 0.751559	 Accuracy:60.417%
Epoch : 1 [64/171 (36%)]	Loss: 0.629498	 Accuracy:66.250%
Epoch : 1 [96/171 (55%)]	Loss: 0.660346	 Accuracy:67.857%
Epoch : 1 [128/171 (73%)]	Loss: 0.670158	 Accuracy:68.056%
Epoch : 1 [110/171 (91%)]	Loss: 0.560989	 Accuracy:65.909%
Epoch : 2 [0/171 (0%)]	Loss: 0.624824	 Accuracy:68.750%
Epoch : 2 [32/171 (18%)]	Loss: 0.593675	 Accuracy:70.833%
Epoch : 2 [64/171 (36%)]	Loss: 0.510769	 Accuracy:67.500%
Epoch : 2 [96/171 (55%)]	Loss: 0.542457	 Accuracy:65.179%
Epoch : 2 [128/171 (73%)]	Loss: 0.449139	 Accuracy:68.056%
Epoch : 2 [110/

In [78]:
criterion = nn.BCELoss()

In [79]:
model_1.eval()
predict_1 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_1(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_1.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.5453603863716125, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [80]:
criterion = nn.BCELoss()

In [81]:
model_2.eval()
predict_2 = []
#y_val = val_y

correct_val = 0.
with torch.no_grad():
    val_loss_summary = 0.0
    for j, val in enumerate(val_loader):
        val_X, val_y = val['X'], val['y']
        if use_cuda:
            val_X, val_y = val_X.cuda(), val_y.cuda()
        val_X, val_y = Variable(val_X).float(), Variable(val_y).float()
        val_pred = model_2(val_X)
        val_pred = val_pred.type_as(torch.FloatTensor())
        for i in val_pred:
            predict_2.append(i.numpy())
        val_loss = criterion(val_pred, val_y)
        val_loss_summary += val_loss
        predicted = torch.max(val_pred.data, 1)[1] 
        correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

******************** Test ********************
Loss: 0.586574375629425, Accuracy: 0.7291666865348816 %
**********************************************
Val accuracy:0.729


In [69]:
predict_1

[array([0.39712372], dtype=float32),
 array([0.20998392], dtype=float32),
 array([0.32124352], dtype=float32),
 array([0.34580824], dtype=float32),
 array([0.43426463], dtype=float32),
 array([0.43412134], dtype=float32),
 array([0.39131775], dtype=float32),
 array([0.38226905], dtype=float32),
 array([0.42771617], dtype=float32),
 array([0.2733672], dtype=float32),
 array([0.42271107], dtype=float32),
 array([0.4628345], dtype=float32),
 array([0.4519838], dtype=float32),
 array([0.3947924], dtype=float32),
 array([0.39574006], dtype=float32),
 array([0.4404809], dtype=float32),
 array([0.3974393], dtype=float32),
 array([0.48102662], dtype=float32),
 array([0.42520103], dtype=float32),
 array([0.43733662], dtype=float32),
 array([0.3043311], dtype=float32),
 array([0.4567577], dtype=float32),
 array([0.36690694], dtype=float32),
 array([0.38817948], dtype=float32),
 array([0.5215086], dtype=float32),
 array([0.4337628], dtype=float32),
 array([0.40996504], dtype=float32),
 array([0.6

In [70]:
predict_2

[array([0.40936574], dtype=float32),
 array([0.30449998], dtype=float32),
 array([0.37039232], dtype=float32),
 array([0.37293124], dtype=float32),
 array([0.45267642], dtype=float32),
 array([0.44899222], dtype=float32),
 array([0.41215253], dtype=float32),
 array([0.40887272], dtype=float32),
 array([0.4442977], dtype=float32),
 array([0.39285985], dtype=float32),
 array([0.4360027], dtype=float32),
 array([0.48552424], dtype=float32),
 array([0.46931365], dtype=float32),
 array([0.40514487], dtype=float32),
 array([0.41182923], dtype=float32),
 array([0.42791894], dtype=float32),
 array([0.405836], dtype=float32),
 array([0.4981986], dtype=float32),
 array([0.43440622], dtype=float32),
 array([0.44471112], dtype=float32),
 array([0.37168515], dtype=float32),
 array([0.47441038], dtype=float32),
 array([0.3651678], dtype=float32),
 array([0.40067512], dtype=float32),
 array([0.5199345], dtype=float32),
 array([0.4233134], dtype=float32),
 array([0.4351335], dtype=float32),
 array([0.

In [71]:
#predict_3 = predict_1 + predict_2

In [72]:
len(predict_3)

NameError: name 'predict_3' is not defined

In [56]:
type(predict_1)

list

In [92]:
import operator

predict_3 = map(operator.add, predict_1, predict_2)

In [93]:
print(predict_3)

In [94]:
predict = []

for i in predict_3:
    print (i/2)
    predict.append(i/2)
    print (predict)



[0.43645966]
[array([0.43645966], dtype=float32)]
[0.34106642]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32)]
[0.4075713]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32)]
[0.40122908]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32)]
[0.44226646]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32)]
[0.44394046]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32)]
[0.41578606]
[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0

In [97]:
print(predict)
type(predict)

[array([0.43645966], dtype=float32), array([0.34106642], dtype=float32), array([0.4075713], dtype=float32), array([0.40122908], dtype=float32), array([0.44226646], dtype=float32), array([0.44394046], dtype=float32), array([0.41578606], dtype=float32), array([0.41548812], dtype=float32), array([0.4436235], dtype=float32), array([0.357646], dtype=float32), array([0.43614274], dtype=float32), array([0.4559007], dtype=float32), array([0.44684273], dtype=float32), array([0.42056525], dtype=float32), array([0.42621326], dtype=float32), array([0.42557824], dtype=float32), array([0.4176554], dtype=float32), array([0.45774207], dtype=float32), array([0.43780085], dtype=float32), array([0.4308337], dtype=float32), array([0.4050982], dtype=float32), array([0.4504068], dtype=float32), array([0.39789623], dtype=float32), array([0.41764146], dtype=float32), array([0.4750628], dtype=float32), array([0.4315881], dtype=float32), array([0.4379266], dtype=float32), array([0.4739846], dtype=float32), arra

list

In [104]:
val_y_pd_ = val_y_pd.to_numpy()
val_y_pd_ = torch.from_numpy(val_y_pd_)
val_y_pd_ = val_y_pd_.type_as(torch.FloatTensor())

In [ ]:
val_loss = criterion(predict, val_y)
val_loss_summary += val_loss
predicted = torch.max(val_pred.data, 1)[1] 
correct_val += (predicted == val_y).sum()

print('*'*20, 'Test', '*'*20)
print('Loss: {}, Accuracy: {} %'.format(val_loss.item(), correct_val/(len(val_loader)*batch_size)))
print('*'*46)
print("Val accuracy:{:.3f}".format(float(correct_val) / (len(val_loader)*batch_size)))

In [96]:
val_y_pd

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     0.0
6     0.0
7     0.0
8     1.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    0.0
15    0.0
16    0.0
17    1.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    1.0
27    0.0
28    0.0
29    1.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    1.0
37    1.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
Name: label, dtype: float64

In [93]:
#num_ftrs = model.forward.in_features
 
for name, param in model.named_parameters():
    print (name, param.data)

layer1.0.weight tensor([[-0.0565, -0.1205,  0.0296,  ...,  0.2230, -0.0025,  0.0114],
        [-0.0879, -0.0202, -0.0235,  ...,  0.0587,  0.0370, -0.0111],
        [-0.0381, -0.0511, -0.0141,  ..., -0.0142,  0.0329,  0.0550],
        ...,
        [ 0.0349, -0.0490, -0.0410,  ..., -0.0243, -0.0495,  0.0048],
        [ 0.0487,  0.1016, -0.0085,  ..., -0.1036,  0.0225, -0.0505],
        [-0.0129,  0.1087,  0.0454,  ..., -0.1197,  0.0375,  0.0101]])
layer1.0.bias tensor([-0.0296, -0.0339,  0.0066, -0.0166, -0.0347, -0.0489,  0.0233,  0.0351,
         0.0348,  0.0116,  0.0404, -0.0197, -0.0154,  0.0126,  0.0236,  0.0361,
        -0.0178, -0.0001, -0.0028, -0.0274, -0.0495, -0.0317,  0.0020,  0.0040,
         0.0019,  0.0077,  0.0134, -0.0460,  0.0494,  0.0400, -0.0072, -0.0119,
         0.0044, -0.0115,  0.0237, -0.0051, -0.0136, -0.0101,  0.0195,  0.0419,
        -0.0333, -0.0286,  0.0231, -0.0157,  0.0131,  0.0034, -0.0038,  0.0113,
        -0.0342,  0.0022, -0.0403,  0.0045,  0.0368, -0.

In [95]:
model.layer1 = 

NameError: name 'Sequential' is not defined

In [79]:
nn.Linear(num_ftrs)

TypeError: __init__() missing 1 required positional argument: 'out_features'

In [32]:
list(model.children())[0:3]

AttributeError: 'list' object has no attribute 'parameter'

## Train model

In [77]:
model = MLPRegressor()

NameError: name 'X_features' is not defined

In [78]:
criterion = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

NameError: name 'model' is not defined

In [44]:
batch_size = 32

In [45]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

val = data_utils.TensorDataset(val_X, val_y)
val_loader = data_utils.DataLoader(val, batch_size=batch_size, shuffle=False)

NameError: name 'val_X' is not defined

In [13]:
trn_X_pd = df2_mod.iloc[:,2:]

In [14]:
trn_X_pd

ID,ENSG00000069482,ENSG00000072657,ENSG00000078399,ENSG00000080572,ENSG00000100678,ENSG00000104435,ENSG00000104888,ENSG00000105146,ENSG00000109321,ENSG00000112499,...,ENSG00000274576,ENSG00000275756,ENSG00000276775,ENSG00000277247,ENSG00000278196,ENSG00000278698,ENSG00000279834,ENSG00000279970,ENSG00000280411,ENSG00000281880
TCGA-61-1910,0.970585,0.003561,0.044476,0.000000,0.000000,0.000000,0.043732,2.308592,1.014494,0.000000,...,0.000000,0.920597,0.000000,1.604301,0.299376,0.000000,0.894232,1.612154,0.761720,0.000000
TCGA-24-0982,0.395276,0.005010,0.090384,0.007062,0.000000,0.006438,0.082034,0.242661,0.691196,0.004156,...,24.797152,0.259035,4.625172,0.188089,4.211878,0.270091,0.034947,0.311310,0.357217,0.000000
TCGA-36-1580,1.403594,0.008641,0.133845,0.001970,0.002559,0.041762,0.074384,0.984290,2.329158,0.001391,...,4.017149,0.289083,0.893372,0.314861,4.817979,0.000000,0.000000,0.707253,15.996040,0.000000
TCGA-25-1321,0.308600,0.003129,0.004478,0.162223,0.000788,0.000000,0.101272,0.240456,1.740308,0.004819,...,2.576079,0.166843,0.000000,0.121147,1.356422,0.521892,0.157563,2.033770,1.495528,0.000000
TCGA-30-1892,0.206074,0.133216,0.119614,0.056072,0.000000,0.055381,0.280458,0.592872,0.914734,0.004400,...,1.058604,0.114269,2.825069,0.000000,3.716018,0.000000,0.000000,0.294278,1.890974,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-04-1347,0.319288,0.000491,0.033696,0.005704,0.000988,0.011701,0.060742,0.663423,0.094262,0.000671,...,0.000000,0.000000,0.000000,3.038608,0.567030,0.000000,0.000000,0.071847,4.616723,0.000000
TCGA-24-0968,0.237050,0.009708,0.185222,0.000000,0.000000,0.000000,0.307004,0.213121,1.291373,0.003796,...,15.221570,0.525782,4.332951,2.720169,31.845646,0.205584,0.000000,0.710875,6.117782,0.000000
TCGA-30-1891,0.772034,0.000481,0.110194,0.000000,0.000646,0.200678,0.000000,0.098616,2.959271,0.077290,...,47.331214,0.000000,2.631492,0.000000,65.568652,0.000000,0.000000,0.000000,80.395413,0.000000
TCGA-09-0366,0.195435,0.000085,1.631170,0.026487,0.001376,0.006339,0.303848,0.703636,5.424154,0.008418,...,0.225023,0.534375,0.100086,1.481522,3.712525,0.000000,0.058985,2.477118,0.602935,0.001573


In [15]:
trn_y_pd = df2_mod.iloc[:,1]

In [16]:
trn_y_pd

TCGA-61-1910    0.007666
TCGA-24-0982    0.007190
TCGA-36-1580    0.027081
TCGA-25-1321    0.002315
TCGA-30-1892    0.057091
                  ...   
TCGA-04-1347    0.000000
TCGA-24-0968    0.013682
TCGA-30-1891    0.034186
TCGA-09-0366    0.032361
TCGA-13-0884    0.018597
Name: ENSG00000048545, Length: 214, dtype: float64

In [17]:
trn_X = torch.from_numpy(trn_X_pd.astype(float).values)
trn_y = torch.from_numpy(trn_y_pd.astype(float).values)

In [18]:
batch_size=64

In [19]:
trn = data_utils.TensorDataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)

In [20]:
tmp = next(iter(trn_loader))

In [21]:
# for dictionary batch
class Dataset(data_utils.Dataset):
   
    def __init__(self, X, y):
        self.X = X
        self.y = y
   
    def __getitem__(self, idx):
        return {'X': self.X[idx], 'y': self.y[idx]}
   
    def __len__(self):
        return len(self.X)

In [22]:
trn = Dataset(trn_X, trn_y)
trn_loader = data_utils.DataLoader(trn, batch_size=batch_size, shuffle=True)